In [1]:
## KAGGLE ONLY
# from shutil import copyfile
# copyfile(src="../input/inputs/generate_dataloaders.py", dst="../working/generate_dataloaders.py")
# copyfile(src="../input/inputs/train_dataloader.p", dst="../working/train_dataloader.p")
# copyfile(src="../input/inputs/val_dataloader.p", dst="../working/val_dataloader.p")
# copyfile(src="../input/inputs/centroids_dataloader.p", dst="../working/ground_truth_dataloader.p")
# copyfile(src="../input/inputs/dictionary.p", dst="../working/dictionary.p")

In [3]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

#from datasets import get_mnist_dataset, get_data_loader
#from utils import *
#from models import *

import pickle as pkl
import os
import datetime as dt
import pandas as pd
import random

from generate_dataloaders import *
from tqdm import tqdm_notebook as tqdm

import evaluation
import importlib
importlib.reload(evaluation)

<module 'evaluation' from '/Users/elliotsilva/Desktop/DS-GA-1006/FairFrame/evaluation.py'>

## Get Dataloaders

In [4]:
seed = 1029
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.
torch.manual_seed(seed)
torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def _init_fn(worker_id):
    np.random.seed(int(seed))

In [5]:
path = os.getcwd()
data_dir = path + '/'
data_dir = path +'/data/' #Uncomment for local system

#### *Verify filenames are consistent*

In [6]:
train_loader = pkl.load(open(data_dir + 'train_dataloader.p','rb'))
train_loader_labelled = pkl.load(open(data_dir + 'train_labeled_dataloader.p','rb'))
train_loader_unlabelled = pkl.load(open(data_dir + 'train_unlabeled_dataloader.p','rb'))
val_loader = pkl.load(open(data_dir + 'val_dataloader.p','rb'))

In [7]:
review_dict = pkl.load(open(data_dir + 'dictionary.p','rb'))

In [8]:
#%conda install pytorch torchvision -c pytorch
## if torch.__version__ is not 1.3.1, run this cell then restart kernel

In [9]:
print(torch.__version__)

1.3.1


## PRE TRAINED WORD EMBEDDINGS 

In [10]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float16')

In [11]:
def load_embeddings(path):
    with open(path) as f:
        return dict(get_coefs(*line.strip().split(' ')) for line in tqdm(f))

In [12]:
def build_matrix(review_dict, embedding_index ,dim = 200):
#     embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(review_dict.tokens), dim))
    unknown_words = []
    
    for word, i in review_dict.ids.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            unknown_words.append(word)
    return embedding_matrix, unknown_words

In [13]:
#glove_twitter = '../input/glove-global-vectors-for-word-representation/glove.twitter.27B.200d.txt' #Change loc for local system
glove_twitter = data_dir + 'glove.twitter.27B.200d.txt'

In [14]:
embedding_index = load_embeddings(glove_twitter)

In [15]:
glove_embedding_index,unknown_words = build_matrix(review_dict, embedding_index)
del embedding_index

In [16]:
len(review_dict.tokens)

16256

In [17]:
len(unknown_words)

4428

In [18]:
# for word in unknown_words:
#     print(word)

## Neural Network Class

NOTE: Data loader is defined as:
- tuple: (tokens, flagged_index, problematic)

In [27]:
class neuralNetBow_glove(nn.Module):
    """
    BagOfWords classification model
    """
    # NOTE: we can't use linear layer until we take weighted average, otherwise it will
    # remember certain positions incorrectly (ie, 4th word has bigger weights vs 7th word)
    def __init__(self, opts):
        super(neuralNetBow_glove, self).__init__()
        self.embedding_matrix = opts['embedding_matrix']
        self.vocab_size = self.embedding_matrix.shape[0]
        self.embed_size = self.embedding_matrix.shape[1]
        self.upweight = opts['upweight']
        
        self.embed = nn.Embedding(self.vocab_size, self.embed_size, padding_idx=0)
        self.embed.weight = nn.Parameter(torch.tensor(self.embedding_matrix,dtype=torch.float32))
        self.embed.weight.requires_grad = False
    
    def forward(self, tokens, flagged_index):
        batch_size, num_tokens = tokens.shape
        embedding = self.embed(tokens)
        
        # upweight by flagged_index
        embedding[torch.LongTensor(range(batch_size)),flagged_index.type(torch.LongTensor),:] *= self.upweight
        
        # average across embeddings
        embedding_ave = embedding.sum(1) / (num_tokens + self.upweight - 1)
        
        return embedding_ave

### Clustering Stuff

In [20]:
class KMeansCriterion(nn.Module):
    
    def __init__(self):
        super().__init__()
    
    def forward(self, embeddings, centroids):
        distances = torch.sum((embeddings[:, None, :] - centroids)**2, 2)
        cluster_distances, cluster_assignments = distances.min(1)
        loss = cluster_distances.sum()
        return loss, cluster_assignments

In [21]:
def centroid_init(k, d, dataloader, model, current_device):
    ## Here we ideally don't want to do randomized/zero initialization
    centroid_sums = torch.zeros(k, d).to(current_device)
    centroid_counts = torch.zeros(k).to(current_device)
    for (tokens, labels, flagged_indices) in dataloader:
        # cluster_assignments = torch.LongTensor(tokens.size(0)).random_(k)
        cluster_assignments = labels.to(current_device)
        
        model.eval()
        sentence_embed = model(tokens.to(current_device),flagged_indices.to(current_device))
    
        update_clusters(centroid_sums, centroid_counts,
                        cluster_assignments, sentence_embed.to(current_device))
    
    centroid_means = centroid_sums / centroid_counts[:, None].to(current_device)
    return centroid_means.clone()

def update_clusters(centroid_sums, centroid_counts,
                    cluster_assignments, embeddings):
    k = centroid_sums.size(0)

    centroid_sums.index_add_(0, cluster_assignments, embeddings)
    bin_counts = torch.bincount(cluster_assignments,minlength=k).type(torch.FloatTensor).to(current_device)
    centroid_counts.add_(bin_counts)
    
    #np_cluster_assignments = cluster_assignments.to('cpu')
    #np_counts = np.bincount(np_cluster_assignments.data.numpy(), minlength=k)
    #centroid_counts.add_(torch.FloatTensor(np_counts))

## Training Function

In [41]:
def train_model(model, centroids, criterion, optimizer, train_loader, valid_loader, num_frozen_epochs=10, num_unfrozen_epochs=0, path_to_save=None, print_every = 1000):

    train_losses=[]
    val_losses=[]
    num_gpus = torch.cuda.device_count()
    if num_gpus > 0:
        current_device = 'cuda'
    else:
        current_device = 'cpu'
    
    unfrozen = False
    for epoch in range(num_frozen_epochs + num_unfrozen_epochs):
        if epoch >= num_frozen_epochs:
            print("*** UNFREEZING MODEL ***")
            model.embed.weight.requires_grad = True
            unfrozen = True
        print('{} | Epoch {}'.format(dt.datetime.now(), epoch))
        model.train()
        k, d = centroids.size()
        centroid_sums = torch.zeros_like(centroids).to(current_device)
        centroid_counts = torch.zeros(k).to(current_device)
        total_epoch_loss = 0

        # run one epoch of gradient descent on autoencoders wrt centroids
        for i, (tokens, labels, flagged_indices) in tqdm(enumerate(train_loader)):
            tokens = tokens.to(current_device)
            labels = labels.to(current_device)
            flagged_indices = flagged_indices.to(current_device)

            # forward pass and compute loss
            sentence_embed = model(tokens,flagged_indices)
            cluster_loss, cluster_assignments = criterion(sentence_embed, centroids.detach())

            # run update step
            optimizer.zero_grad()
            if unfrozen:
                cluster_loss.backward()
            optimizer.step()
            
            #Add loss to the epoch loss
            total_epoch_loss += cluster_loss.data

            # store centroid sums and counts in memory for later centering
            update_clusters(centroid_sums, centroid_counts,
                            cluster_assignments, sentence_embed)

            if i % print_every == 0:
                losses = cluster_loss.data/len(tokens)
                print('Average training loss at batch ',i,': %.3f' % losses)
            
        total_epoch_loss /= len(train_loader.dataset)
        train_losses.append(total_epoch_loss)
        print('Average training loss after epoch ',epoch,': %.3f' % total_epoch_loss)
        
        # update centroids based on assignments from autoencoders
        centroids = centroid_sums / (centroid_counts[:, None] + 1).to(current_device)
        
        # calculate validation loss after every epoch
        total_validation_loss = 0
        for i, (tokens, labels, flagged_indices) in enumerate(valid_loader):
            model.eval()
            tokens = tokens.to(current_device)
            labels = labels.to(current_device)
            flagged_indices = flagged_indices.to(current_device)
            
            # forward pass and compute loss
            sentence_embed = model(tokens,flagged_indices)
            cluster_loss, cluster_assignments = criterion(sentence_embed, centroids)
            
            #Add loss to the validation loss
            total_validation_loss += cluster_loss.data

        total_validation_loss /= len(valid_loader.dataset)
        val_losses.append(total_validation_loss)
        print('Average validation loss after epoch ',epoch,': %.3f' % total_validation_loss)
        
        if path_to_save == None:
            pass
        else:
            opts = {"embedding_matrix":model.embedding_matrix} #change options depending on model inputs required
            torch.save(model.state_dict(), path_to_save+'model_dict.pt')
            torch.save(centroids, path_to_save+'centroids')
            torch.save(train_losses, path_to_save+'train_losses')
            torch.save(val_losses, path_to_save+'val_losses')
            torch.save(opts, path_to_save+'opts')
            
        
    return model, centroids, train_losses, val_losses

In [42]:
num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    current_device = 'cuda'
else:
    current_device = 'cpu'

# Hyperparameter Tuning Loop

In [43]:
def get_save_directory(opts):
    path = os.getcwd()
    model_folder = 'baseline_randomized_embeddings/'
    model_dir = path + '/models/' + model_folder
    
    # subfolder for each hyperparam config
    num_unfrozen_epochs = opts['num_unfrozen_epochs']
    upweight = opts['upweight']
    subfolder = "num_unfrozen_epochs="+str(num_unfrozen_epochs) + ",upweight="+str(upweight) + '/'
    
    # need to actually create these subfolders lol
    try:
        os.makedirs(model_dir + subfolder) # will throw error if subfolder already exists
    except:
        pass
    
    return model_dir + subfolder

In [44]:
def train_config(opts):
    model = neuralNetBow_glove(opts).to(current_device)
    num_unfrozen_epochs = opts['num_unfrozen_epochs']
    centroids = centroid_init(2, 200, train_loader_labelled, model, current_device)
    criterion = KMeansCriterion().to(current_device)
    optimizer = torch.optim.Adam(model.parameters(), 0.01, amsgrad=True)
    path_to_save = get_save_directory(opts)
    print(path_to_save)
    
    train_model(model, centroids, criterion, optimizer, train_loader, val_loader, num_frozen_epochs=10, num_unfrozen_epochs=num_unfrozen_epochs, path_to_save=path_to_save)

In [45]:
upweights = [1, 5, 10, 25]
num_unfrozen_epochs_list = [0, 1]

for num_unfrozen_epochs in num_unfrozen_epochs_list:
    for upweight in upweights:
        opts = {
            'embedding_matrix': glove_embedding_index,
            'num_unfrozen_epochs': num_unfrozen_epochs,
            'upweight': upweight
        }
        train_config(opts)

/Users/elliotsilva/Desktop/DS-GA-1006/FairFrame/models/baseline_randomized_embeddings/num_unfrozen_epochs=0,upweight=1/
2019-12-07 21:32:33.565856 | Epoch 0


Average training loss at batch  0 : 0.881
Average training loss at batch  1000 : 1.239
Average training loss at batch  2000 : 1.010
Average training loss at batch  3000 : 1.009

Average training loss after epoch  0 : 1.176
Average validation loss after epoch  0 : 0.708
2019-12-07 21:32:36.306309 | Epoch 1


Average training loss at batch  0 : 0.474
Average training loss at batch  1000 : 0.662
Average training loss at batch  2000 : 0.659
Average training loss at batch  3000 : 0.836

Average training loss after epoch  1 : 0.659
Average validation loss after epoch  1 : 0.704
2019-12-07 21:32:39.299727 | Epoch 2


Average training loss at batch  0 : 0.521
Average training loss at batch  1000 : 0.543
Average training loss at batch  2000 : 0.597
Average training loss at batch  3000 : 0.610

Average training loss after epoch  2 : 0.658
Average validation loss after epoch  2 : 0.702
2019-12-07 21:32:41.946083 | Epoch 3


Average training loss at batch  0 : 0.575
Average training loss at batch  1000 : 0.502
Average training loss at batch  2000 : 0.677
Average training loss at batch  3000 : 0.677

Average training loss after epoch  3 : 0.657
Average validation loss after epoch  3 : 0.700
2019-12-07 21:32:44.681060 | Epoch 4


...

Average training loss at batch  0 : 3.592
Average training loss at batch  1000 : 0.088
Average training loss at batch  2000 : 0.062
Average training loss at batch  3000 : 0.020

Average training loss after epoch  10 : 0.216
Average validation loss after epoch  10 : 0.041
/Users/elliotsilva/Desktop/DS-GA-1006/FairFrame/models/baseline_randomized_embeddings/num_unfrozen_epochs=2,upweight=1/
2019-12-07 21:43:09.660214 | Epoch 0


Average training loss at batch  0 : 0.872
Average training loss at batch  1000 : 1.087
Average training loss at batch  2000 : 1.221
Average training loss at batch  3000 : 1.561

Average training loss after epoch  0 : 1.176
Average validation loss after epoch  0 : 0.708
2019-12-07 21:43:12.548812 | Epoch 1


Average training loss at batch  0 : 0.747
Average training loss at batch  1000 : 0.495
Average training loss at batch  2000 : 0.787
Average training loss at batch  3000 : 0.636

Average training loss after epoch  1 : 0.659
Average validation loss after epoch  1 : 0.704
2019-12-07 21:43:15.306159 | Epoch 2


Average training loss at batch  0 : 0.820
Average training loss at batch  1000 : 0.750
Average training loss at batch  2000 : 0.664


KeyboardInterrupt: 

## Evaluate Model

To run this by itself, run:
- Get Dataloaders
- class definitions (model, clustering)

In [46]:
num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    current_device = 'cuda'
else:
    current_device = 'cpu'

In [49]:
def evaluate_config(opts,verbose=True):
    path_to_save = get_save_directory(opts)
    print(path_to_save)
    
    model = neuralNetBow_glove(opts) #change according to model inputs
    model.load_state_dict(torch.load(path_to_save+'model_dict.pt',map_location=lambda storage, loc: storage))
    model = model.to(current_device)
    criterion = KMeansCriterion().to(current_device)
    centroids = torch.load(path_to_save+'centroids',map_location=lambda storage, loc: storage)
    
    TP_cluster, FP_cluster, results_dict = evaluation.main(model, centroids, val_loader, criterion, data_dir, current_device, verbose)
    results_dict.update(opts)
    return TP_cluster, FP_cluster, results_dict

In [52]:
upweights = [1, 5, 10, 25]
num_unfrozen_epochs_list = [2, 5, 10]

results_df = pd.DataFrame()
for num_unfrozen_epochs in num_unfrozen_epochs_list:
    for upweight in upweights:
        opts = {
            'embedding_matrix': glove_embedding_index,
            'num_unfrozen_epochs': num_unfrozen_epochs,
            'upweight': upweight            
        }
        _, _, results_dict = evaluate_config(opts,False)
        results_df = results_df.append(results_dict,ignore_index=True)
        
results_df = results_df[['num_unfrozen_epochs','upweight','Accuracy','F1 score','Precision','Recall',
                        'TP_rate','FP_rate','FN_rate','TN_rate']]

/Users/elliotsilva/Desktop/DS-GA-1006/FairFrame/models/baseline_randomized_embeddings/num_unfrozen_epochs=0,upweight=1/
/Users/elliotsilva/Desktop/DS-GA-1006/FairFrame/models/baseline_randomized_embeddings/num_unfrozen_epochs=0,upweight=5/
/Users/elliotsilva/Desktop/DS-GA-1006/FairFrame/models/baseline_randomized_embeddings/num_unfrozen_epochs=0,upweight=10/
/Users/elliotsilva/Desktop/DS-GA-1006/FairFrame/models/baseline_randomized_embeddings/num_unfrozen_epochs=0,upweight=25/
/Users/elliotsilva/Desktop/DS-GA-1006/FairFrame/models/baseline_randomized_embeddings/num_unfrozen_epochs=1,upweight=1/
/Users/elliotsilva/Desktop/DS-GA-1006/FairFrame/models/baseline_randomized_embeddings/num_unfrozen_epochs=1,upweight=5/
/Users/elliotsilva/Desktop/DS-GA-1006/FairFrame/models/baseline_randomized_embeddings/num_unfrozen_epochs=1,upweight=10/
/Users/elliotsilva/Desktop/DS-GA-1006/FairFrame/models/baseline_randomized_embeddings/num_unfrozen_epochs=1,upweight=25/


In [53]:
results_df

,num_unfrozen_epochs,upweight,Accuracy,F1 score,Precision,Recall,TP_rate,FP_rate,FN_rate,TN_rate
0,0.0,1.0,0.563356,0.677236,0.916185,0.537144,0.916185,0.083815,0.789474,0.210526
1,0.0,5.0,0.559714,0.674785,0.913545,0.534968,0.913545,0.086455,0.794118,0.205882
2,0.0,10.0,0.557551,0.673527,0.912791,0.533646,0.912791,0.087209,0.797688,0.202312
3,0.0,25.0,0.563652,0.683933,0.944206,0.536143,0.944206,0.055794,0.816901,0.183099
4,1.0,1.0,0.508683,0.640279,0.874510,0.505015,0.874510,0.125490,0.857143,0.142857
5,1.0,5.0,0.604410,0.688776,0.875486,0.567704,0.875486,0.124514,0.666667,0.333333
6,1.0,10.0,0.563107,0.695946,1.000000,0.533679,1.000000,0.000000,0.873786,0.126214
7,1.0,25.0,0.539094,0.665337,0.916318,0.522283,0.916318,0.083682,0.838129,0.161871
